# **Machine Learning Logistic Regression Model**


---------------
## **I. Problem Statement**

### **Context**

**A significant number of hotel bookings are called off due to cancellations or no-shows.** Typical reasons for cancellations include change of plans, scheduling conflicts, etc. This is often made easier by the option to do so free of charge or preferably at a low cost. This may be beneficial to hotel guests, but it is a less desirable and possibly revenue-diminishing factor for hotels to deal with. Such losses are particularly high on last-minute cancellations.

The new technologies involving online booking channels have dramatically changed customers’ booking possibilities and behavior. This adds a further dimension to the challenge of how hotels handle cancellations, which are no longer limited to traditional booking and guest characteristics.

This pattern of cancellations of bookings impacts a hotel on various fronts:
1. **Loss of resources (revenue)** when the hotel cannot resell the room.
2. **Additional costs of distribution channels** by increasing commissions or paying for publicity to help sell these rooms.
3. **Lowering prices last minute**, so the hotel can resell a room, resulting in reducing the profit margin.
4. **Human resources to make arrangements** for the guests.

### **Objective**

This increasing number of cancellations calls for a Machine Learning based solution that can help in predicting which booking is likely to be canceled. INN Hotels Group has a chain of hotels in Portugal - they are facing problems with this high number of booking cancellations and have reached out to your firm for data-driven solutions. You, as a Data Scientist, have to analyze the data provided to find which factors have a high influence on booking cancellations, build a predictive model that can predict which booking is going to be canceled in advance, and help in formulating profitable policies for cancellations and refunds.


### **Data Description**

The data contains the different attributes of customers' booking details. The detailed data dictionary is given below:


**Data Dictionary**

* **Booking_ID:** Unique identifier of each booking
* **no_of_adults:** Number of adults
* **no_of_children:** Number of children
* **no_of_weekend_nights:** Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel
* **no_of_week_nights:** Number of weekday nights (Monday to Friday) the guest stayed or booked to stay at the hotel
* **type_of_meal_plan:** Type of meal plan booked by the customer:
    * Not Selected – No meal plan selected
    * Meal Plan 1 – Breakfast
    * Meal Plan 2 – Half board (breakfast and one other meal)
    * Meal Plan 3 – Full board (breakfast, lunch, and dinner)
* **required_car_parking_space:** Does the customer require a car parking space? (0 - No, 1- Yes)
* **room_type_reserved:** Type of room reserved by the customer. The values are ciphered (encoded) by INN Hotels.
* **lead_time:** Number of days between the date of booking and the arrival date
* **arrival_year:** Year of arrival date
* **arrival_month:** Month of arrival date
* **arrival_date:** Date of the month
* **market_segment_type:** Market segment designation.
* **repeated_guest:** Is the customer a repeated guest? (0 - No, 1- Yes)
* **no_of_previous_cancellations:** Number of previous bookings that were canceled by the customer prior to the current booking
* **no_of_previous_bookings_not_canceled:** Number of previous bookings not canceled by the customer prior to the current booking
* **avg_price_per_room:** Average price per day of the reservation; prices of the rooms are dynamic. (in euros)
* **no_of_special_requests:** Total number of special requests made by the customer (e.g. high floor, view from the room, etc)
* **booking_status:** Flag indicating if the booking was canceled or not.

## **II. Importing the libraries required**

## **Loading the dataset**

In [ ]:
# Importing the basic libraries we will require for the project

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

# Code to ignore warnings from function usage
import warnings;

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importing the Machine Learning models we require from Scikit-Learn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

warnings.filterwarnings('ignore')
sns.set()
sns.set_theme(style="whitegrid")


In [ ]:
data = pd.read_csv("../data/reservations-db.csv")

In [ ]:
# Copying data to another variable to avoid any changes to original data
reservations_db = data.copy()

## **III. Overview of the dataset**

### **View the first and last 5 rows of the dataset**

Let's **view the first few rows and last few rows** of the dataset in order to understand its structure a little better.

We will use the head() and tail() methods from Pandas to do this.

In [ ]:
reservations_db.head()

In [ ]:
reservations_db.tail()

### **Understand the shape of the dataset**

In [ ]:
reservations_db.shape

* The dataset has 36275 rows and 19 columns. 

### **Check the data types of the columns for the dataset**

In [ ]:
reservations_db.info()

* `Booking_ID`, `type_of_meal_plan`, `room_type_reserved`, `market_segment_type`, and `booking_status` are of object type while rest columns are numeric in nature.

* There are no null values in the dataset.

### **Dropping duplicate values**

In [ ]:
# checking for duplicate values
reservations_db.duplicated().sum()

- There are **no duplicate values** in the data.

### **Dropping the unique values column**

**Let's drop the Booking_ID column first before we proceed forward**, as a column with unique values will have almost no predictive power for the Machine Learning problem at hand.

In [ ]:
reservations_db = reservations_db.drop(["Booking_ID"], axis=1)

In [ ]:
reservations_db.head()

### **1: Check the summary statistics of the dataset and write your observations**



**Let's check the statistical summary of the data.**

In [ ]:
reservations_db.describe() 

**Observations:**

- Guests most often arrives as individual or couples. few families.
- appear to be business travels as there are very few children other than what appears to an isolated case
- guests book short stays; typically around 2 day. no_of_weekend_nights appears to have some bad data (max==7 is impossible)
- ditto with no_of_week_nights. max==17 is impossible.
- most guests do arrive with a car
- guests appear to most book reservations just ahead of their arrival date. the bottom 25 percentile cover nearly 20% of reservation, and more than half are covered with the next 2 months. there's wide dispersion overall though, so anywhere from just a few days in advance to up to 6 months.
- our data set covers around 18 months
- most guests are visiting for the first time. very few repeat guests
- no_of_previous_cancellations might have some bad data. no_of_previous_cancellations==13 seems unusual.
- no_of_previous_bookings_not_canceled also might have some bad data
- avg_price_per_room suggests that INN Hotels offer basic lodging given the avg price of 103 with std dev of only 35
- no_of_special_requests looks interesting. the upper 75 percentile might offer useful information.


## **IV. Exploratory Data Analysis**

### **2: Univariate Analysis**

Let's explore these variables in some more depth by observing their distributions.

We will first define a **hist_box() function** that provides both a boxplot and a histogram in the same visual, with which we can perform univariate analysis on the columns of this dataset.

In [ ]:
# Defining the hist_box() function
def hist_box(data,col):
  kwargs = {
      'height_ratios': (0.15, 0.85)
      }
  f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw=kwargs, figsize=(12,6))
  # Adding a graph in each part
  sns.boxplot(data[col], ax=ax_box, showmeans=True)
  sns.distplot(data[col], ax=ax_hist)

  ax_box.set_title(col + ' Histogram')
  ax_box.set_xlabel(col)

  plt.show()

#### **2.1:  Plot the histogram and box plot for the variable `Lead Time` using the hist_box function provided and write your insights.**

In [ ]:
hist_box(reservations_db, "lead_time") 

**Observations:**

Most guests are booking reservations just ahead of their arrival date, with the remainder being kind of normally distributed. Nearly all reservations are made within around six months of arrival, and virtually no one books more than 1 year in advance.

#### **2.2:  Plot the histogram and box plot for the variable `Average Price per Room` using the hist_box function provided and write your insights.**

In [ ]:
hist_box(reservations_db, "avg_price_per_room")

- Management comps a lot of rooms (the bump at $0)
- most rooms rates range from around 75 euros to 125 euros
- there are few rate options below 75 euros
- INN Hotels offers some REALLY expensive rooms; as high as 300 euros in extreme cases. there are few takers though.
- there appears to be a "most guests prefer" rate option given the spike in the middle of around 100 euros

**Interestingly some rooms have a price equal to 0. Let's check them.**

In [ ]:
reservations_db[reservations_db["avg_price_per_room"] == 0]

- There are quite a few hotel rooms which have a price equal to 0.
- In the market segment column, it looks like many values are complementary.

In [ ]:
reservations_db.loc[reservations_db["avg_price_per_room"] == 0, "market_segment_type"].value_counts()

* It makes sense that most values with room prices equal to 0 are the rooms given as complimentary service from the hotel.
* The rooms booked online must be a part of some promotional campaign done by the hotel.

In [ ]:
# Calculating the 25th quantile
Q1 = data["avg_price_per_room"].quantile(0.25)

# Calculating the 75th quantile
Q3 = data["avg_price_per_room"].quantile(0.75)

# Calculating IQR
IQR = Q3 - Q1

# Calculating value of upper whisker
Upper_Whisker = Q3 + 1.5 * IQR
Upper_Whisker

In [ ]:
# assigning the outliers the value of upper whisker
reservations_db.loc[reservations_db["avg_price_per_room"] >= 500, "avg_price_per_room"] = Upper_Whisker

#### **Let's understand the distribution of the categorical variables**

**Number of Children**

In [ ]:
sns.histplot(data=reservations_db['no_of_children'])
plt.show()

In [ ]:
reservations_db['no_of_children'].value_counts(normalize=True)

* Customers were not travelling with children in 93% of cases.
* There are some values in the data where the number of children is 9 or 10, which is highly unlikely. 
* We will replace these values with the maximum value of 3 children.

In [ ]:
# replacing 9, and 10 children with 3
reservations_db["no_of_children"] = reservations_db["no_of_children"].replace([9, 10], 3)

**Arrival Month**

In [ ]:
sns.histplot(reservations_db["arrival_month"])
plt.show()

In [ ]:
reservations_db['arrival_month'].value_counts(normalize=True)

* October is the busiest month for hotel arrivals followed by September and August. **Over 35% of all bookings**, as we see in the above table, were for one of these three months.
* Around 14.7% of the bookings were made for an October arrival.

**Booking Status**

In [ ]:
sns.histplot(data = reservations_db["booking_status"])
plt.show()

In [ ]:
reservations_db['booking_status'].value_counts(normalize=True)

* 32.8% of the bookings were canceled by the customers.

In [ ]:
reservations_db["booking_status"] = reservations_db["booking_status"].apply(
    lambda x: 1 if x == "Canceled" else 0
)

### **3: Bivariate Analysis**

#### **3.1: Find and visualize the correlation matrix using a heatmap.**



In [ ]:
cols_list = reservations_db.select_dtypes(include=np.number).columns.tolist()

plt.figure(figsize=(12, 7))
sns.heatmap(reservations_db.corr())
plt.show()

**Observations:**

relationships of primary interest:
- no_of_previous_bookings_not_canceled vs repeated_guest
- no_of_previous_bookings_not_canceled vs no_of_previous_cancellations
- booking_status vs lead_time

relationships of secondary interest:
- avg_price_per_room vs no_of_adults
- avg_price_per_room vs no_of_children
- no_of_previous_cancellations vs repeated_guest


**Hotel rates are dynamic and change according to demand and customer demographics. Let's see how prices vary across different market segments**

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=reservations_db, x="market_segment_type", y="avg_price_per_room", palette="gist_rainbow"
)
plt.show()

* Rooms booked online have high variations in prices.
* The offline and corporate room prices are almost similar.
* Complementary market segment gets the rooms at very low prices, which makes sense.

We will define a **stacked barplot()** function to help analyse how the target variable varies across predictor categories.

In [ ]:
# Defining the stacked_barplot() function
def stacked_barplot(data,predictor,target,figsize=(10,6)):
  (pd.crosstab(data[predictor],reservations_db[target],normalize='index')*100).plot(kind='bar',figsize=figsize,stacked=True)
  plt.legend(loc="lower right")
  plt.ylabel('Percentage Cancellations %')

#### **3.2: Plot the stacked barplot for the variable `Market Segment Type` against the target variable `Booking Status` using the stacked_barplot  function provided and write your insights.**

In [ ]:
stacked_barplot(reservations_db, "market_segment_type", "booking_status")

**Observations:**
- online reservations offer the most potential for improvement, with a 38%ish percent cancellation rate
- Aviation and offline are tied at around 35%, but aviation sounds like a category where the guests are not in control of their schedules, and so the cancellations might not be unavoidable.
- corporate might not be a worthwhile effort given that there are few cancellations.
- no cancellations for Complimentary rooms. lets disregard hereon.

#### **3.3: Plot the stacked barplot for the variable `Repeated Guest` against the target variable `Booking Status` using the stacked_barplot  function provided and write your insights.**

Repeating guests are the guests who stay in the hotel often and are important to brand equity.

In [ ]:
stacked_barplot(reservations_db, "repeated_guest", "booking_status")

**Observations:**
- Repeat guests almost never cancel. we can remove them from consideration
- nonrepeat guests cancel nearly 40% of the time.


**Let's analyze the customer who stayed for at least a day at the hotel.**

In [ ]:
stay_data = reservations_db[(reservations_db["no_of_week_nights"] > 0) & (reservations_db["no_of_weekend_nights"] > 0)]
stay_data["total_days"] = (stay_data["no_of_week_nights"] + stay_data["no_of_weekend_nights"])

stacked_barplot(stay_data, "total_days", "booking_status",figsize=(15,6))

* The general trend is that the chances of cancellation increase as the number of days the customer planned to stay at the hotel increases.

**As hotel room prices are dynamic, Let's see how the prices vary across different months**

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(y=reservations_db["avg_price_per_room"], x=reservations_db["arrival_month"], ci=None)
plt.show()

* The price of rooms is highest in May to September - around 115 euros per room.

## **V. Data Preparation for Modeling**

- We want to predict which bookings will be canceled.
- Before we proceed to build a model, we'll have to encode categorical features.
- We'll split the data into train and test to be able to evaluate the model that we build on the train data.

**Separating the independent variables (X) and the dependent variable (Y)**

In [ ]:
X = reservations_db.drop(["booking_status"], axis=1)
Y = reservations_db["booking_status"]

X = pd.get_dummies(X, drop_first=True) # Encoding the Categorical features

**Splitting the data into a 70% train and 30% test set**

Some classification problems can exhibit a large imbalance in the distribution of the target classes: for instance there could be several times more negative samples than positive samples. In such cases it is recommended to use the **stratified sampling** technique to ensure that relative class frequencies are approximately preserved in each train and validation fold.

In [ ]:
# Splitting data in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30,stratify=Y, random_state=1)

In [ ]:
print("Shape of Training set : ", X_train.shape)
print("Shape of test set : ", X_test.shape)
print("Percentage of classes in training set:")
print(y_train.value_counts(normalize=True))
print("Percentage of classes in test set:")
print(y_test.value_counts(normalize=True))

## **VI. Model Evaluation Criterion**

#### **Model can make wrong predictions as:**

1. Predicting a customer will not cancel their booking but in reality, the customer will cancel their booking.
2. Predicting a customer will cancel their booking but in reality, the customer will not cancel their booking. 

#### **Which case is more important?** 

Both the cases are important as:

* If we predict that a booking will not be canceled and the booking gets canceled then the hotel will lose resources and will have to bear additional costs of distribution channels.

* If we predict that a booking will get canceled and the booking doesn't get canceled the hotel might not be able to provide satisfactory services to the customer by assuming that this booking will be canceled. This might damage brand equity. 



#### **How to reduce the losses?**

* The hotel would want the `F1 Score` to be maximized, the greater the F1  score, the higher the chances of minimizing False Negatives and False Positives. 

**Also, let's create a function to calculate and print the classification report and confusion matrix so that we don't have to rewrite the same code repeatedly for each model.**

In [ ]:
# Creating metric function 
def metrics_score(actual, predicted):
    print(classification_report(actual, predicted))

    cm = confusion_matrix(actual, predicted)
    plt.figure(figsize=(8,5))
    
    sns.heatmap(cm, annot=True,  fmt='.2f', xticklabels=['Not Cancelled', 'Cancelled'], yticklabels=['Not Cancelled', 'Cancelled'])
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

## **VII. Building the model**

### **4: Logistic Regression**

#### **4.1: Build a Logistic Regression model (Use the sklearn library)**

In [ ]:
# Fitting logistic regression model
lg=LogisticRegression()
lg.fit(X_train, y_train)

#### **4.2: Check the performance of the model on train and test data**

In [ ]:
# Checking the performance on the training data
y_pred_train = lg.predict(X_train)
metrics_score(y_train, y_pred_train)

**Observations:**
- True Negative (Actual=0, Predicted=0): Model predicts that a guest would not cancel and the guest does not cancel.
- False Positive (Actual=0, Predicted=1): Model predicts that a guest would cancel but the guest does not cancel.
- False Negative (Actual=1, Predicted=0): Model predicts that a guest would not cancel but the guest cancels.
- True Positive (Actual=1, Predicted=1): Model predicts that a guest would cancel and the guest actually cancels.

with an overall F1 (accuracy score) of 0.80 the model is good but not great.

Let's check the performance on the test set

In [ ]:
# Checking the performance on the test dataset
y_pred_test = lg.predict(X_test)
metrics_score(y_test, y_pred_test)

**Observations:**
- **We are getting an accuracy of around 80%** on train and test dataset.
- However, **the recall for this model is only around 57% for class 1 on train and 66% on test (see below).**
- As the recall is low, **this model will not perform well** in differentiating out those guests who have a high chance of cancelling their reservation, meaning it will eventually not help in reducing the cancellation rate. 
- As we can see from the Confusion Matrix, **this model does a mediocre job of identifying the guests who are at risk of cancellation.**


#### **4.3: Find the optimal threshold for the model using the Precision-Recall Curve.**

Precision-Recall curves summarize the trade-off between the true positive rate and the positive predictive value for a predictive model using different probability thresholds.

Let's use the Precision-Recall curve and see if we can find a **better threshold.**


In [ ]:
# Predict_proba gives the probability of each observation belonging to each class
y_scores_lg=lg.predict_proba(X_train)

precisions_lg, recalls_lg, thresholds_lg = precision_recall_curve(y_train, y_scores_lg[:,1])


# Plot values of precisions, recalls, and thresholds
plt.figure(figsize=(10,7))
plt.plot(thresholds_lg, precisions_lg[:-1], 'b--', label='precision')
plt.plot(thresholds_lg, recalls_lg[:-1], 'g--', label = 'recall')
plt.xlabel('Threshold')
plt.legend(loc='upper left')
plt.ylim([0,1])
plt.show()

**Observations**
- precision and recall are balanced for a threshold of about ~0.42

In [ ]:
# Setting the optimal threshold
optimal_threshold = 0.42

#### **4.4: Check the performance of the model on train and test data using the optimal threshold.**

In [ ]:
# Creating confusion matrix
y_pred_train = lg.predict_proba(X_train)
metrics_score(y_train, y_pred_train[:,1]>optimal_threshold)

**Observations:**
- recall has increased from 0.60 to 0.68 for class 1
- recall has decreased slightly from 0.90 to 0.85 for class 2

Let's check the performance on the test set

In [ ]:
y_pred_test = lg.predict_proba(X_test)
metrics_score(y_test, y_pred_test[:,1]>optimal_threshold)

**Observations:**

- The model is demonstrates similar performance on the test and training data.
- The recall of the test data has increased somewhat, from 0.60 to 0.67. while at the same time, the precision has decreased from 0.73 to 0.68.
- The average recall and precision for the model remain good but not great. 


## **Happy Learning!**